In [1]:
using RigidBodyDynamics
using RigidBodySim
using MeshCatMechanisms
using StrandbeestRobot

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/998a51f82a651317b2614fbfe83ea114061d742e-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/998a51f82a651317b2614fbfe83ea114061d742e-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

┌ Info: Recompiling stale cache file /home/rdeits/.julia/compiled/v1.0/StrandbeestRobot/Nyqao.ji for StrandbeestRobot [f3f0ac65-cc51-52c1-880d-2041552cfde2]
└ @ Base loading.jl:1184


In [2]:
mechanism = StrandbeestRobot.mechanism(floating=false, add_flat_ground=false)

Spanning tree:
Vertex: world (root)
  Vertex: crossbar, Edge: crossbar_to_world
    Vertex: crank_axle, Edge: joint_crossbar_crank
      Vertex: pair01_bar_m, Edge: pair01_joint_crank_axle_m
        Vertex: pair01_leg1_bar_j, Edge: pair01_leg1_joint_m_j
          Vertex: pair01_leg1_bar_k, Edge: pair01_leg1_joint_j_k
            Vertex: pair01_leg1_bar_c, Edge: pair01_leg1_joint_k_c
            Vertex: pair01_leg1_bars_g_h_i, Edge: pair01_leg1_joint_k_i
          Vertex: pair01_leg1_bars_b_d_e, Edge: pair01_leg1_joint_j_e
            Vertex: pair01_leg1_bar_f, Edge: pair01_leg1_joint_e_f
        Vertex: pair01_leg2_bar_j, Edge: pair01_leg2_joint_m_j
          Vertex: pair01_leg2_bar_k, Edge: pair01_leg2_joint_j_k
            Vertex: pair01_leg2_bar_c, Edge: pair01_leg2_joint_k_c
            Vertex: pair01_leg2_bars_g_h_i, Edge: pair01_leg2_joint_k_i
          Vertex: pair01_leg2_bars_b_d_e, Edge: pair01_leg2_joint_j_e
            Vertex: pair01_leg2_bar_f, Edge: pair01_leg2_joint_e_f
 

In [3]:
gui = GUI(mechanism, URDFVisuals(StrandbeestRobot.urdfpath()))

In [ ]:
#NBSKIP
open(gui)

In [7]:
state = MechanismState(mechanism)
StrandbeestRobot.solve_initial_configuration!(state)
copyto!(gui.visualizer, state)

┌ Warning: Initial position cannot be on the boundary of the box. Moving elements to the interior.
│ Element indices affected: [14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 44, 45, 47, 48, 50, 51, 53, 54, 56, 57, 59, 60, 62, 63, 65, 66, 68, 69, 71, 72]
└ @ Optim /home/rdeits/.julia/packages/Optim/fabGe/src/multivariate/solvers/constrained/fminbox.jl:212


In [8]:
# Simulate the robot
control! = let I = velocity_range(state, findjoint(mechanism, "joint_crossbar_crank"))
    function (τ, t, state)
        τ[I] .= -10.0 .* (velocity(state)[I] .- 3.0)
    end
end
dynamics = Dynamics(mechanism, control!)
problem = ODEProblem(dynamics, state, (0, 5.0))
@time solution = solve(problem, Tsit5(), abs_tol=1e-6, dt=1e-6)
setanimation!(gui.visualizer, solution)

  6.746012 seconds (3.91 M allocations: 324.230 MiB, 1.79% gc time)
